In [7]:
import common

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
import wandb

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
model_checkpoint_filename_list = [
                                    'model_bak/cp-efficientnet_b3.ra2_in1k_sd_42_epc_2_isFull_False_vl0.0596_va_0.9802_vf1_0.9795.pt', 
                                    'model_bak/cp-tiny_vit_21m_384.dist_in22k_ft_in1k_sd_42_epc_0_isFull_False_vl_0.1706_va_0.9366_vf1_0.9290.pt'
                                 ]
is_soft_voting = True

In [10]:

model_list = []
tst_loader_list = []

for cp_filename in model_checkpoint_filename_list:
    checkpoint = torch.load(cp_filename, map_location = device)
    
    print(checkpoint['model'], checkpoint['tst_img_size'], checkpoint['batch_size'])
    
    model = timm.create_model(
        checkpoint['model'],
        pretrained = True,
        num_classes = 17,
    ).to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])    
    model_list.append(model)
    
    ##
    tst_transform = common.create_tst_transform(checkpoint['tst_img_size'])

    tst_dataset = common.ImageDataset(
        "datasets_fin/sample_submission.csv",
        "datasets_fin/test/",
        transform = tst_transform
    )

    tst_loader = DataLoader(
        tst_dataset,
        batch_size = checkpoint['batch_size'],
        shuffle = False,
        num_workers = 2,
        pin_memory = True
    )
    
    tst_loader_list.append(tst_loader)

print(len(model_list), len(tst_loader_list))

efficientnet_b3.ra2_in1k 320 32
tiny_vit_21m_384.dist_in22k_ft_in1k 384 32


model.safetensors:   0%|          | 0.00/85.0M [00:00<?, ?B/s]

2 2


c:\Users\jjam\anaconda3\envs\pytorch_test3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jjam\.cache\huggingface\hub\models--timm--tiny_vit_21m_384.dist_in22k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
preds_list = []

for model, tst_loader in zip(model_list, tst_loader_list):
    preds = common.get_preds_list_by_tst_loader(model, tst_loader, device, is_soft_voting)
    preds_list.append(preds)

100%|██████████| 99/99 [00:43<00:00,  2.26it/s]


In [12]:
# hard voting 구현
def hard_voting(predictions):
    predictions = np.asarray(predictions)
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

def soft_voting(predictions):
    predictions = np.asarray(predictions)
    mean_axis0 = np.mean(predictions, axis=0)
    return mean_axis0.argmax(axis=1)

# 최종 예측
if is_soft_voting:
    final_pred = soft_voting(preds_list)
else:
    final_pred = hard_voting(preds_list)

# csv 로 저장
common.preds_list_to_save_to_csv(final_pred, tst_loader, 'pred_ensemble.csv')